In [48]:
from datetime import date
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine

In [49]:
# Configuración de las conexiones a las bases de datos
with open('./config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDO_Y_FURIOSO']
    config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [50]:
fact_estados_servicio = pd.read_sql_table('mensajeria_estadosservicio', ryf)

fact_servicio = pd.read_sql_table('fact_servicio', etl_conn)
dim_estado = pd.read_sql_table('dim_estado', etl_conn)
dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)

In [51]:
fact_estados_servicio.head()

,id,fecha,hora,foto,observaciones,estado_id,servicio_id,es_prueba,foto_binary
0,1014,2024-01-29,01:13:32,foto,4 tubos,4,226,False,None
1,1484,2024-01-30,18:45:12,foto,Demora,3,79,True,None
2,2829,2024-02-06,11:34:04,foto,Compra exitosa,5,613,False,None
3,1888,2024-02-01,14:50:39,foto,Zzxzz,4,376,False,None
4,32312,2024-04-06,16:11:21,foto,No,3,7164,True,None


In [52]:
fact_estados_servicio['hora'] = fact_estados_servicio['hora'].astype(str).str.split('.').str[0]
fact_estados_servicio['datetime_estado'] = pd.to_datetime(fact_estados_servicio['fecha'].astype(str) + ' ' + fact_estados_servicio['hora'].astype(str))

In [53]:
fact_estados_servicio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128402 entries, 0 to 128401
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               128402 non-null  int64         
 1   fecha            128402 non-null  datetime64[ns]
 2   hora             128402 non-null  object        
 3   foto             128402 non-null  object        
 4   observaciones    128401 non-null  object        
 5   estado_id        128402 non-null  int64         
 6   servicio_id      128402 non-null  int64         
 7   es_prueba        128402 non-null  bool          
 8   foto_binary      270 non-null     object        
 9   datetime_estado  128402 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](2), int64(3), object(4)
memory usage: 8.9+ MB


In [54]:
fact_estados_servicio = fact_estados_servicio.merge(dim_fecha[['key_dim_fecha', 'fecha']])
fact_estados_servicio = fact_estados_servicio.merge(dim_estado[['key_dim_estado', 'estado_id']])
fact_estados_servicio = fact_estados_servicio.merge(fact_servicio[['key_fact_servicio', 'servicio_id']])

In [56]:
fact_estados_servicio['tiempo_en_estado'] = fact_estados_servicio.groupby('servicio_id')['datetime_estado'].diff()
fact_estados_servicio['minutos_en_estado'] = fact_estados_servicio['tiempo_en_estado'].dt.total_seconds() / 60

fact_tiempos_estado = fact_estados_servicio.groupby([
    'key_dim_fecha',
    'estado_id',
]).agg({
    'minutos_en_estado': ['sum', 'mean', 'count'],
    'servicio_id': 'nunique'
}).reset_index()

fact_tiempos_estado = fact_tiempos_estado.rename(columns={
    'minutos_en_estado_sum': 'total_minutos',
    'minutos_en_estado_mean': 'promedio_minutos',
    'minutos_en_estado_count': 'cantidad_transiciones',
    'servicio_id': 'cantidad_servicios'
})

In [57]:
fact_tiempos_estado.head()

key_dim_fecha estado_id minutos_en_estado                    \
                                        sum        mean count   
0           253         1          0.000000         NaN     0   
1           256         2      -2686.333333 -335.791667     8   
2           257         1          0.000000         NaN     0   
3           260         1          0.000000         NaN     0   
4           267         1          0.000000         NaN     0   

  cantidad_servicios  
             nunique  
0                 12  
1                  6  
2                  8  
3                  1  
4                  1

In [ ]:
fact_estados_servicio.replace({
    np.nan: 'no aplica',
    'NaT': 'no aplica'
}, inplace=True)

In [ ]:
fact_estados_servicio['saved'] = date.today()

In [ ]:
fact_estados_servicio.to_sql(
    'fact_estados_servicio',
    etl_conn,
    if_exists='replace',
    index=False
)

402